In [7]:

"""MobileNet v2 models for Keras.
# Reference
- [Inverted Residuals and Linear Bottlenecks Mobile Networks for
   Classification, Detection and Segmentation]
   (https://arxiv.org/abs/1801.04381)
"""


from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout, DepthwiseConv2D
from keras.layers import Activation, BatchNormalization, add, Reshape
# from keras.applications.mobilenet import relu6, 
from keras.utils.vis_utils import plot_model

from keras import backend as K

def relu6(x):
    return K.relu(x, max_value=6)

def _conv_block(inputs, filters, kernel, strides):
    """Convolution Block
    This function defines a 2D convolution operation with BN and relu6.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    return Activation(relu6)(x)


def _bottleneck(inputs, filters, kernel, t, s, r=False):
    """Bottleneck
    This function defines a basic bottleneck structure.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        r: Boolean, Whether to use the residuals.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    tchannel = K.int_shape(inputs)[channel_axis] * t

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation(relu6)(x)

    x = Conv2D(filters, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if r:
        x = add([x, inputs])
    return x


def _inverted_residual_block(inputs, filters, kernel, t, strides, n):
    """Inverted Residual Block
    This function defines a sequence of 1 or more identical layers.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        n: Integer, layer repeat times.
    # Returns
        Output tensor.
    """

    x = _bottleneck(inputs, filters, kernel, t, strides)

    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, 1, True)

    return x


def MobileNetv2(input_shape, k):
    """MobileNetv2
    This function defines a MobileNetv2 architectures.
    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        k: Integer, number of classes.
    # Returns
        MobileNetv2 model.
    """

    inputs = Input(shape=input_shape)
    x = _conv_block(inputs, 32, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=1, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, strides=2, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, strides=1, n=3)
    x = _inverted_residual_block(x, 160, (3, 3), t=6, strides=2, n=3)
    x = _inverted_residual_block(x, 320, (3, 3), t=6, strides=1, n=1)

    x = _conv_block(x, 1280, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, 1280))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)

    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)
    plot_model(model, to_file='images/MobileNetv2.png', show_shapes=True)

    return model


mb = MobileNetv2((224, 224, 3), 100)

In [ ]:
[((None, 224, 224, 3), 'input_2'),
 ((None, 225, 225, 3), 'conv1_pad'),
 ((None, 112, 112, 32), 'conv1'),
 ((None, 112, 112, 32), 'conv1_bn'),
 ((None, 112, 112, 32), 'conv1_relu'),
 
 ((None, 112, 112, 32), 'conv_dw_1'),
 ((None, 112, 112, 32), 'conv_dw_1_bn'),
 ((None, 112, 112, 32), 'conv_dw_1_relu'),
 ((None, 112, 112, 64), 'conv_pw_1'),
 ((None, 112, 112, 64), 'conv_pw_1_bn'),
 ((None, 112, 112, 64), 'conv_pw_1_relu'),
 
 ((None, 113, 113, 64), 'conv_pad_2'),
 ((None, 56, 56, 64), 'conv_dw_2'),
 ((None, 56, 56, 64), 'conv_dw_2_bn'),
 ((None, 56, 56, 64), 'conv_dw_2_relu'),
 ((None, 56, 56, 128), 'conv_pw_2'),
 ((None, 56, 56, 128), 'conv_pw_2_bn'),
 ((None, 56, 56, 128), 'conv_pw_2_relu'),
 
 ((None, 56, 56, 128), 'conv_dw_3'),
 ((None, 56, 56, 128), 'conv_dw_3_bn'),
 ((None, 56, 56, 128), 'conv_dw_3_relu'),
 ((None, 56, 56, 128), 'conv_pw_3'),
 ((None, 56, 56, 128), 'conv_pw_3_bn'),
 ((None, 56, 56, 128), 'conv_pw_3_relu'),
 
 ((None, 57, 57, 128), 'conv_pad_4'),
 
 ((None, 28, 28, 128), 'conv_dw_4'),
 ((None, 28, 28, 128), 'conv_dw_4_bn'),
 ((None, 28, 28, 128), 'conv_dw_4_relu'),
 ((None, 28, 28, 256), 'conv_pw_4'),
 ((None, 28, 28, 256), 'conv_pw_4_bn'),
 ((None, 28, 28, 256), 'conv_pw_4_relu'),
 
 ((None, 28, 28, 256), 'conv_dw_5'),
 ((None, 28, 28, 256), 'conv_dw_5_bn'),
 ((None, 28, 28, 256), 'conv_dw_5_relu'),
 ((None, 28, 28, 256), 'conv_pw_5'),
 ((None, 28, 28, 256), 'conv_pw_5_bn'),
 ((None, 28, 28, 256), 'conv_pw_5_relu'),
 
 ((None, 29, 29, 256), 'conv_pad_6'),
 
 ((None, 14, 14, 256), 'conv_dw_6'),
 ((None, 14, 14, 256), 'conv_dw_6_bn'),
 ((None, 14, 14, 256), 'conv_dw_6_relu'),
 ((None, 14, 14, 512), 'conv_pw_6'),
 ((None, 14, 14, 512), 'conv_pw_6_bn'),
 ((None, 14, 14, 512), 'conv_pw_6_relu'),
 
 ((None, 14, 14, 512), 'conv_dw_7'),
 ((None, 14, 14, 512), 'conv_dw_7_bn'),
 ((None, 14, 14, 512), 'conv_dw_7_relu'),
 ((None, 14, 14, 512), 'conv_pw_7'),
 ((None, 14, 14, 512), 'conv_pw_7_bn'),
 ((None, 14, 14, 512), 'conv_pw_7_relu'),
 
 ((None, 14, 14, 512), 'conv_dw_8'),
 ((None, 14, 14, 512), 'conv_dw_8_bn'),
 ((None, 14, 14, 512), 'conv_dw_8_relu'),
 ((None, 14, 14, 512), 'conv_pw_8'),
 ((None, 14, 14, 512), 'conv_pw_8_bn'),
 ((None, 14, 14, 512), 'conv_pw_8_relu'),
 
 ((None, 14, 14, 512), 'conv_dw_9'),
 ((None, 14, 14, 512), 'conv_dw_9_bn'),
 ((None, 14, 14, 512), 'conv_dw_9_relu'),
 ((None, 14, 14, 512), 'conv_pw_9'),
 ((None, 14, 14, 512), 'conv_pw_9_bn'),
 ((None, 14, 14, 512), 'conv_pw_9_relu'),
 
 ((None, 14, 14, 512), 'conv_dw_10'),
 ((None, 14, 14, 512), 'conv_dw_10_bn'),
 ((None, 14, 14, 512), 'conv_dw_10_relu'),
 ((None, 14, 14, 512), 'conv_pw_10'),
 ((None, 14, 14, 512), 'conv_pw_10_bn'),
 ((None, 14, 14, 512), 'conv_pw_10_relu'),
 
 ((None, 14, 14, 512), 'conv_dw_11'),
 ((None, 14, 14, 512), 'conv_dw_11_bn'),
 ((None, 14, 14, 512), 'conv_dw_11_relu'),
 ((None, 14, 14, 512), 'conv_pw_11'),
 ((None, 14, 14, 512), 'conv_pw_11_bn'),
 ((None, 14, 14, 512), 'conv_pw_11_relu'),
 
 ((None, 15, 15, 512), 'conv_pad_12'),
 
 ((None, 7, 7, 512), 'conv_dw_12'),
 ((None, 7, 7, 512), 'conv_dw_12_bn'),
 ((None, 7, 7, 512), 'conv_dw_12_relu'),
 ((None, 7, 7, 1024), 'conv_pw_12'),
 ((None, 7, 7, 1024), 'conv_pw_12_bn'),
 ((None, 7, 7, 1024), 'conv_pw_12_relu'),
 
 ((None, 7, 7, 1024), 'conv_dw_13'),
 ((None, 7, 7, 1024), 'conv_dw_13_bn'),
 ((None, 7, 7, 1024), 'conv_dw_13_relu'),
 ((None, 7, 7, 1024), 'conv_pw_13'),
 ((None, 7, 7, 1024), 'conv_pw_13_bn'),
 ((None, 7, 7, 1024), 'conv_pw_13_relu'),
 
 ((None, 1024), 'global_average_pooling2d_2'),
 ((None, 1, 1, 1024), 'reshape_1'),
 ((None, 1, 1, 1024), 'dropout'),
 ((None, 1, 1, 1000), 'conv_preds'),
 ((None, 1, 1, 1000), 'act_softmax'),
 ((None, 1000), 'reshape_2')]

In [8]:
mb.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
import keras

mb_k = keras.applications.mobilenet.MobileNet(weights=None, input_shape=(224,224,3))

In [13]:
mb_k.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [22]:
len(mb.layers), len(mb_k.layers), mb_k.layers[-1].output.get_shape()

(159, 93, TensorShape([Dimension(None), Dimension(1000)]))

In [27]:
plot_model(mb_k, to_file='MobileNetv2_orig.png', show_shapes=True)
plot_model(mb  , to_file='MobileNetv2_mine.png', show_shapes=True)


In [35]:
from keras.layers import Conv2D,SeparableConv2D
from keras.layers import Flatten
from keras.layers.merge import concatenate

# from ssd_layers import PriorBox

import os
import warnings

from keras.models import Model
from keras.layers import Input, Activation, Reshape, Dropout, Conv2D, DepthwiseConv2D, ZeroPadding2D
from keras.layers import BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine import get_source_inputs
from keras.engine.base_layer import InputSpec
from keras.applications import imagenet_utils
from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K


def relu6(x):
    return K.relu(x, max_value=6)


def preprocess_input(x):

    return imagenet_utils.preprocess_input(x, mode='tf')


def _conv_block(inputs, filters, alpha, kernel=(3, 3), strides=(1, 1)):
    
    channel_axis = 3
    filters = int(filters * alpha)
    x = ZeroPadding2D(padding=(1, 1), name='conv1_pad')(inputs)
    x = Conv2D(filters, kernel,padding='valid',use_bias=False,strides=strides,name='conv1')(x)
    x = BatchNormalization(axis=channel_axis, name='conv1_bn')(x)
    return Activation(relu6, name='conv1_relu')(x)


def _depthwise_conv_block(inputs, pointwise_conv_filters, alpha,
                          depth_multiplier=1, strides=(1, 1), block_id=1):
    
    channel_axis = 3# if K.image_data_format() == 'channels_first' else -1
    pointwise_conv_filters = int(pointwise_conv_filters * alpha)

    x = ZeroPadding2D(padding=(1, 1), name='conv_pad_%d' % block_id)(inputs)
    x = DepthwiseConv2D((3, 3),padding='valid',depth_multiplier=depth_multiplier,strides=strides,use_bias=False,name='conv_dw_%d' % block_id)(x)
    x = BatchNormalization(axis=channel_axis, name='conv_dw_%d_bn' % block_id)(x)
    x = Activation(relu6, name='conv_dw_%d_relu' % block_id)(x)
    x = Conv2D(pointwise_conv_filters, (1, 1),padding='same',use_bias=False,strides=(1, 1),name='conv_pw_%d' % block_id)(x)
    x = BatchNormalization(axis=channel_axis, name='conv_pw_%d_bn' % block_id)(x)
    return Activation(relu6, name='conv_pw_%d_relu' % block_id)(x)

def _depthwise_conv_block_f(inputs, depth_multiplier=1, strides=(1, 1), block_id=1):
    channel_axis = 1
    x = ZeroPadding2D(padding=(1, 1), name='conv_pad_%d'  % block_id)(inputs)
    x = DepthwiseConv2D((3, 3),padding='valid',depth_multiplier=depth_multiplier,strides=strides,use_bias=False,name='conv_dw_%d' % block_id)(x)
    x = BatchNormalization(axis=channel_axis, name='conv_dw_%d_bn' % block_id)(x)
    return Activation(relu6, name='conv_dw_%d_relu' % block_id)(x)


def _conv_blockSSD_f(inputs, filters, alpha, kernel, strides,block_id=11):
    channel_axis = 3
    filters = int(filters * alpha)
    Conv = Conv2D(filters, kernel,padding='valid',use_bias=False,strides=strides,name='conv__%d' % block_id)(inputs)
    x = BatchNormalization(axis=channel_axis, name='conv_%d_bn' % block_id)(Conv)
    return Activation(relu6, name='conv_%d_relu' % block_id)(x),Conv

def _conv_blockSSD(inputs, filters, alpha,block_id=11):
    channel_axis = 3
    filters = int(filters * alpha)
    x = ZeroPadding2D(padding=(1, 1), name='conv_pad_%d_1' % block_id)(inputs)
    x = Conv2D(filters, (1,1),padding='valid',use_bias=False,strides=(1, 1),name='conv__%d_1'%block_id)(x)
    x = BatchNormalization(axis=channel_axis, name='conv_%d_bn_1'% block_id)(x)
    x = Activation(relu6, name='conv_%d_relu_1'% block_id)(x)
    Conv = Conv2D(filters*2, (3,3), padding='valid', use_bias=False, strides=(2, 2), name='conv__%d_2' % block_id)(x)
    x = BatchNormalization(axis=channel_axis, name='conv_%d_bn_2' % block_id)(Conv)
    x = Activation(relu6, name='conv_%d_relu_2' % block_id)(x)
    return x,Conv

def SSD(input_shape, num_classes):

    img_size=(input_shape[1],input_shape[0])
    input_shape=(input_shape[1],input_shape[0],3)
    alpha = 1.0
    depth_multiplier = 1
    input0 = Input(input_shape)
    x = _conv_block(input0, 32, alpha, strides=(2, 2))
    x = _depthwise_conv_block(x, 64, alpha, depth_multiplier, block_id=1)
    x = _depthwise_conv_block(x, 128, alpha, depth_multiplier,strides=(2, 2), block_id=2)
    x = _depthwise_conv_block(x, 128, alpha, depth_multiplier, block_id=3)
    x = _depthwise_conv_block(x, 256, alpha, depth_multiplier,strides=(2, 2), block_id=4)
    x = _depthwise_conv_block(x, 256, alpha, depth_multiplier, block_id=5)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, strides=(2, 2), block_id=6)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=7)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=8)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=9)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=10)
    x = _depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=11)
    x = _depthwise_conv_block(x, 1024, alpha, depth_multiplier, strides=(2, 2), block_id=12)
    x = _depthwise_conv_block(x, 1024, alpha, depth_multiplier, block_id=13)

#     x = _depthwise_conv_block_f(x, depth_multiplier,strides=(1, 1), block_id=11)
#     x,conv11=_conv_blockSSD_f(x,512,depth_multiplier,kernel=(1, 1), strides=(1, 1),block_id=11)
#     x = _depthwise_conv_block(x, 512, alpha, depth_multiplier,strides=(2, 2), block_id=12)
#     x = _depthwise_conv_block_f(x, depth_multiplier,strides=(1, 1), block_id=13)
#     x, conv13 = _conv_blockSSD_f(x, 512, alpha, kernel=(1, 1), strides=(1, 1), block_id=13)
#     x, conv14_2 = _conv_blockSSD(x, 256, alpha, block_id=14)
#     x, conv15_2 = _conv_blockSSD(x, 128, alpha, block_id=15)
#     x, conv16_2 = _conv_blockSSD(x, 128, alpha, block_id=16)
#     x, conv17_2 = _conv_blockSSD(x, 64, alpha, block_id=17)

    return Model(input0, x)

test = SSD(input_shape=(224,224,3), num_classes=1)

In [36]:
test.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________